In [3]:
import pandas as pd
import requests 
from bs4 import BeautifulSoup

# Do British tabloids treat Meghan Markle more negatively than Kate Middleton? 

I plan to apply natural language processing techniques to investigate whether Meghan Markle has been treated more negatively by British tabloids than Kate Middleton.

To do this, I need to examine articles mentioning Markle and Middleton in key tabloids over the last two years.
I'll use web scraping to collect articles from the largest British tabloids by circulation:
The Daily Mail, The Sun and The Mirror.


# The Daily Mail

### Meghan Markle

Scraping the Daily Mail's website's search feature for links to articles with the search term "Meghan Markle."

In [ ]:
markle_links = []

for i in range(0,6500,50):
    print(i)
    try:
        URL = f"https://www.dailymail.co.uk/home/search.html?offset={i}&size=50&sel=site&searchPhrase=Meghan+Markle&sort=recent&type=article&days=all"
        response = requests.get(URL) 
        soup = BeautifulSoup(response.content) 
        hits = soup.find_all('div', class_='sch-result')
        for hit in hits:
            link_dict = {}
            link_dict['link'] = (hit.find('a').get('href'))
            title = hit.find('div', class_='sch-res-content').find('h3').a.text
            link_dict['title'] = title
            date = hit.find('div', class_='sch-res-content').find('h4').text.strip()
            link_dict['date'] = date
            markle_links.append(link_dict)
    except:
        pass

In [ ]:
df = pd.DataFrame(markle_links)

In [ ]:
df.to_csv('markle_DM_links.csv')

In [3]:
df = pd.read_csv('markle_DM_links.csv')

In [5]:
df[df.title.str.contains('Markle|Meghan')].shape

(3732, 4)

In [7]:
df['author'] = df.date.str.extract(r'(.*)-\n\n')

In [8]:
df['date'] = df.date.str.extract(r'-\n\n(.*)')

In [9]:
df.date = pd.to_datetime(df.date)

In [10]:
df.date.min()

Timestamp('2017-10-20 10:20:32')

In [11]:
df.date.max()

Timestamp('2019-11-28 11:44:39')

In [12]:
df = df[(df['date'] > '2017-11-27') & (df['date'] < '2019-11-27')]

In [13]:
df = df[df.author.str.contains('Mailonline|MailOnline|Dailymail.com|Daily Mail')]

In [14]:
df.shape

(2799, 5)

In [16]:
links = df.link.tolist()

In [17]:
len(links)

2799

In [18]:
stories = []
for link in links:
    print(link)
    try:
        URL = f"https://www.dailymail.co.uk{link}"
        response = requests.get(URL) 
        soup = BeautifulSoup(response.content) 
        story = soup.find('div', itemprop="articleBody").find_all('p', class_=lambda x: x != 'imageCaption')
        url_dict = {}
        url_dict['link'] = link
        lines = []
        for line in story:
            sent = line.text
            lines.append(sent)
        url_dict['article'] = lines
        stories.append(url_dict)
    except:
        print('error', link)

/news/article-7728861/Ex-diplomat-helped-launch-Meghans-career-subjects-broadside-writes-SEBASTIAN-SHAKESPEARE.html
/femail/article-7726927/Tiny-Tickers-charity-reveals-tremendous-support-inspired-Prince-Harry-Meghan-Markle.html
/femail/article-7722387/Meghan-Markle-aware-pitted-against-future-queen-Kate-Middleton-source-claims.html
/femail/article-7722343/Meghan-Markle-Prince-Harry-say-Archie-eating-solids.html
/tvshowbiz/article-7719943/Kim-Kardashian-says-empathises-Meghan-Markle-Prince-Harry.html
/femail/article-7719689/Prince-Harry-Meghan-Markle-thank-inspirational-Sussex-supporters.html
/news/article-7716761/Meghan-Markle-troubled-Prince-Andrew-saying-sex-positive-act.html
/news/article-7706965/Meghan-Markles-uncle-shares-rare-photos-previous-Thanksgivings.html
/femail/article-7715019/Expert-suggests-Prince-William-Kates-stripped-staff-reason-absence.html
/femail/article-7711215/Sussex-supporters-launch-campaign-plant-10-000-trees-Prince-Harry-Meghan-Markles-names.html
/femail/ar

/news/article-7627241/MPs-including-Jess-Phillips-sign-open-leader-backing-Meghan-Markle.html
/news/article-7626087/Princess-Dianas-butler-Paul-Burrell-says-Prince-Harry-Meghan-Markle-US.html
/femail/article-7625383/Kate-Middleton-doing-best-try-reach-Prince-Harry-Meghan-Markle.html
/femail/article-7625103/Windsor-Castle-advertises-new-Housekeeping-Assistant.html
/news/article-7625191/Wendell-Pierce-supports-Meghan-Markle-GMB.html
/sport/sportsnews/article-7623231/Prince-Harry-reveals-fly-Japan-watch-Englands-World-Cup-Final-against-South-Africa.html
/femail/article-7621843/Archbishop-Canterbury-defends-Meghan-Prince-Harry-not-superhuman.html
/femail/article-7612771/Christmas-tree-farm-Prince-Harry-Meghan-Markle-shopped-deliver-one-decorated.html
/femail/article-7621113/Author-Matt-Haig-defends-Meghan-Markle-against-bully-Toby-Young.html
/news/article-7621137/Prince-Harry-Meghan-Markle-considering-creating-second-base-US.html
/news/article-7620265/Alan-Jones-attacks-Meghan-Harry-docume

/femail/article-7553503/Meghan-Markle-seen-beaming-giggling-tries-Instagram-crown-filter.html
/femail/article-7551855/Kim-Kardashian-praises-Prince-Harry-Meghan-Markle-bringing-attention-important-issues.html
/femail/article-7549911/Prince-Harry-Meghan-Markle-removed-Royal-Foundation-website.html
/news/article-7547849/New-NHS-mental-health-website-CRASHES-royal-advert.html
/femail/article-7546875/Prince-Harrys-polo-pal-Nacho-says-defend-life.html
/femail/article-7545521/Archie-enjoyed-views-table-mountain-feeding-surrounded-toys-royal-tour.html
/health/article-7545331/Take-mental-health-test-promoted-Princes-Harry-William-Meghan-Markle-Kate-Middleton.html
/news/article-7544497/Royals-television-takeover-mental-health-campaign.html
/news/article-7543331/Princess-Dianas-former-private-secretary-warns-Harry-Meghan-attack-Press.html
/news/article-7543183/Meghan-Markles-poignant-yellow-ribbon-tribute-student-19-stolen.html
/femail/article-7542829/Meghan-Markle-Prince-Harry-share-touching-hi

/news/article-7490991/Flight-journalists-flying-cover-Meghan-Harry-Africa-delayed-21-HOURS.html
/femail/article-7490469/Local-denim-brand-Duchess-wore-Australian-tour-year-opens-new-facility.html
/news/article-7490419/Prince-Harry-Meghan-Markle-skip-second-day-celebrations-Misha-Nonoos-wedding.html
/news/article-7490423/Meghan-Markle-colouring-books-pens-Archies-baby-clothes-royal-tour.html
/tvshowbiz/article-7490129/Gayle-King-looks-gorgeous-green-gown-Misha-Nonoos-Rome-wedding-alongside-pal-Meghan-Markle.html
/news/article-7489693/Vogue-editor-hits-racist-criticism-Meghan-Markle-Duchess-Sussex-guest-edited.html
/news/article-7489343/Gary-Lineker-defends-Danny-Baker-racist-tweet-got-sacked-BBC-Radio-5-Live.html
/news/article-7489285/Meghan-Markle-Prince-Harry-leave-Rome.html
/news/article-7489021/Exclusive-1-200-night-hotel-Prince-Harry-Meghan-Markle-staying-Rome.html
/news/article-7488769/Prince-Harry-Meghan-Markle-post-tricky-quiz-9-4million-Instagram-account.html
/news/article-7488

/femail/article-7434741/Royal-fans-wild-Harry-Meghans-Force-Change-charity-whale-sharks-Archie.html
/news/article-7434713/Meghan-jets-New-York-watch-friend-Serena-Williams-play.html
/news/article-7433275/Duchess-alarms-Buckingham-Palace-staff-keeping-agent-lawyer-manager-acting-days.html
/femail/article-7430875/Meghan-Markle-Prince-Harry-change-Instagram-post-style-bid-noticed-more.html
/femail/article-7431411/60-Minutes-Australia-slammed-racist-Meghan-Markle-piece.html
/femail/article-7431161/Meghan-Markle-chose-picture-Tonga-previously-unseen-birthday-thank-note-fan.html
/news/article-7430495/Ex-Doctor-star-Alex-Kingston-defends-Meghan-Markle-amid-private-jet-storm.html
/femail/article-7430135/Princess-Charlotte-day-school-Kate-Middleton-looks-elegant.html
/news/article-7429467/Prince-Harry-keeps-pushing-public-away-royal-expert-claims.html
/news/article-7429359/Prince-Harry-Meghan-Markle-ditch-Queens-invite-travel-Balmoral.html
/news/article-7427607/How-Serena-Williams-passed-learne

...

/news/article-5468433/Meghan-never-forgive-Harry-strayed-says-numerologist.html
/news/article-5553995/Harry-Meghan-attend-Invictus-Games-trials-Bath.html
/femail/article-5553565/Prince-Harry-Meghan-Markle-wedding-invite-includes-subtle-nod-divorce.html
/femail/article-5553005/Meghan-Markles-Suits-character-struggles-wedding-plans.html
/news/article-5549535/Football-Association-president-Prince-William-miss-FA-Cup-final.html
/femail/article-5544611/The-Royal-Balcony-unveiled-Madame-Tussauds.html
/femail/article-5544039/Meghan-Markles-nephew-mother-reveal.html
/femail/article-5541327/Prince-Harry-strikes-close-relationship-Thomas-Markle.html
/femail/article-5539497/Meghan-Markles-body-language-shows-charge-relationship.html
/news/article-5539509/Meghan-Markles-father-Thomas-enjoys-dinner-date-mystery-blonde-Mexico.html
/femail/article-5538587/Will-Meghan-Markles-father-royal-coat-arms.html
/news/fb-5536765/Harry-Meghan-enjoy-traditional-Irish-stew-iconic-pub-yes-called-Crown.html
/femail

/tvshowbiz/article-5388463/Sarah-Ferguson-rocks-blue-blazer-night-London.html
/tvshowbiz/article-5388733/SEB-SHAKESPEARE-drama-Harry-Meghan-Royal-Romance.html
/news/article-5388553/Meghan-Markle-pronounces-scone-like-British.html
/femail/article-5387855/Meghan-Markle-embraced-charity-worker-Edinburgh.html
/femail/article-5387093/Meghan-gives-nod-Diana-Burberry-tartan-coat.html
/news/fb-5386539/Meghans-hen-planned-will.html
/news/fb-5386251/How-Meghan-Markle-wore-Black-Watch-tartan-Scottish.html
/femail/article-5385939/Meghan-Markles-425-Strathberry-crossbody-bag-sells-out.html
/femail/article-5385623/How-Meghan-reinventing-royal-dressing.html
/news/article-5385551/GOP-candidate-posted-Markle-picture-banned-Twitter.html
/news/article-5383597/Meghan-Markle-secret-visits-Grenfell-victims-twice.html
/debate/article-5379305/Ann-Widdecombe-blasts-Meghans-CofE-conversion.html
/femail/article-5378525/Archbishop-Canterbury-marry-Harry-Meghan.html
/news/article-5377641/Paul-Nehlen-slammed-racist

/femail/article-5244971/Daniel-Galvin-Jr-designs-Meghans-wedding-tresses.html
/news/article-5244747/Meghan-Markles-father-hails-Harry-gentleman.html
/femail/article-5243909/Meghan-Markles-Christmas-present-Queen-revealed.html
/news/article-5243881/Meghan-Markles-half-brother-blames-brawl-scrutiny.html
/news/article-5240653/Meghan-Markles-sister-law-arrested-assaulted-brother.html
/femail/article-5239647/Meghan-wont-wear-royal-jewels-shes-married.html
/femail/article-5239009/Zara-Mike-Tindall-expecting-second-child.html
/femail/article-5238697/Meghan-Markles-facialist-shares-pre-wedding-routine.html
/femail/article-5234805/Views-Queens-Wikipedia-page-sky-rocket.html
/news/article-5232557/Council-boss-demands-police-clear-Windsor-homeless.html
/femail/article-5231199/How-worlds-hottest-young-royals-spent-New-Years-Eve.html
/femail/article-5231567/Will-Meghans-MOTHER-walk-aisle.html
/femail/article-5228809/Meghan-visit-youth-radio-station-Brixton-week.html
/news/article-5228561/Prince-Har

/femail/article-5140693/George-Charlotte-follow-Meghan-aisle.html
/news/article-5140705/Meghan-stuff-bra-Deal-No-Deal.html
/news/article-5140641/Harry-Meghans-wedding-boost-UK-economy-500m.html
/tvshowbiz/article-5140655/Chrissie-Swan-isnt-fan-Meghan-Markle.html
/femail/article-5138905/Meghans-Scottish-handbag-sells-royal-engagement.html
/femail/article-5137819/Americans-women-British-accent-sexiest.html
/tvshowbiz/article-5137831/Chelsy-Davy-seen-time-Prince-Harrys-news.html
/news/article-5137917/PLATELLS-PEOPLE-Kate-cope-Meghan-mania.html
/femail/article-5137503/Unseen-images-Meghan-Markle-college-graduation-party.html
/news/article-5137911/So-Meghans-dad-determined-hide-world.html
/femail/article-5137387/Meghan-Markles-dream-tour-Europe-teenager.html
/debate/article-5137877/JAN-MOIR-joins-crowds-Meghan-sparkled.html
/femail/article-5137453/Unseen-photos-Meghan-Markle-prom.html
/femail/article-5137785/Meghan-Markle-revelations-friend-knew-best.html
/femail/article-5137445/Meghan-Mark

/tvshowbiz/article-5122523/Patrick-J-Adams-jokes-Meghan-Markle-amid-engagement.html
/tvshowbiz/article-5122481/Will-Pia-Miller-attend-Meghan-Markles-royal-wedding.html
/femail/article-5122335/Spoof-photos-Harry-Meghan-marry-Las-Vegas.html
/tvshowbiz/article-5122449/Cressida-Bonas-parties-amid-Prince-Harry-engagement-news.html
/femail/article-5122033/Meghan-Markles-Beauty-Evolution.html
/news/article-5122359/Meghan-tells-met-Queen-tea.html
/news/article-5122267/Harry-says-Meghan-Diana-best-friends.html
/femail/article-5121971/Meghan-Markles-new-natural-look.html
/femail/article-5121939/Meghan-Markle-engagement-shoes-Aquazzura.html
/tvshowbiz/article-5121747/Khloe-Kardashian-tweets-congratulations-Meghan-Markle.html
/news/article-5121713/PIERS-MORGAN-Congrats-Harry-pal-Meghan-keeper.html
/femail/article-5121291/Who-design-Meghan-Markles-wedding-dress.html
/news/article-5121667/Meghan-Markles-extended-family-comment-engagement.html
/femail/article-5121363/Body-language-expert-reveals-Harr

In [21]:
markle_stories = pd.DataFrame(stories)

In [22]:
markle_stories.to_csv('markle_DM_stories.csv')

I'm cleaning up the dataframe to isolate articles written between May 18, 2017 and May 18 2019 -- the two year period surrounding the date Markle married Prince Harry. While The Daily Mail also publishes articles from wire services, I'm only interested in articles written by Daily Mail reporters.

In [26]:
markle_stories_full = pd.merge(df, markle_stories, on='link')

In [33]:
markle_stories_full.to_csv('markle_DM_stories_Nov17to19.csv')

### Kate Middleton

Scraping the Daily Mail's website's search feature for links to articles with the search term "Kate Middleton."

In [34]:
middleton_links = []

for i in range(0,3500,50):
    try:
        URL = f"https://www.dailymail.co.uk/home/search.html?offset={i}&size=50&sel=site&searchPhrase=Kate+Middleton&sort=recent&type=article&days=all"
        response = requests.get(URL) 
        soup = BeautifulSoup(response.content) 
        hits = soup.find_all('div', class_='sch-result')
        for hit in hits:
            link_dict = {}
            link_dict['link'] = (hit.find('a').get('href'))
            title = hit.find('div', class_='sch-res-content').find('h3').a.text
            link_dict['title'] = title
            date = hit.find('div', class_='sch-res-content').find('h4').text.strip()
            link_dict['date'] = date
            middleton_links.append(link_dict)
    except:
        print('error', i)

In [35]:
df2 = pd.DataFrame(middleton_links)

In [36]:
df2.shape

(3500, 3)

I'm cleaning up the dataframe to isolate articles written between April 29, 2010 and April 29, 2012 -- the two year period surrounding the date Middleton married Prince William. While The Daily Mail also publishes articles from wire services, I'm only interested in articles written by Daily Mail reporters.

In [37]:
df2['author'] = df2.date.str.extract(r'(.*)-\n\n')

In [38]:
df2['date'] = df2.date.str.extract(r'-\n\n(.*)')

In [39]:
df2.date = pd.to_datetime(df2.date)

In [43]:
df2.date.min()

Timestamp('2017-11-27 10:02:04')

In [44]:
df2.date.max()

Timestamp('2019-11-26 20:04:51')

In [42]:
df2 = df2[(df2['date'] > '2017-11-27') & (df2['date'] < '2019-11-27')]

In [45]:
df2.shape

(3003, 4)

In [46]:
df2 = df2[df2.author.str.contains('Mailonline|MailOnline|Dailymail.com|Daily Mail|The Mail')]

In [47]:
df2.shape

(1898, 4)

In [48]:
df2.to_csv('middleton_DM_links.csv')

In [50]:
links2 = df2.link.tolist()

In [51]:
stories2 = []
for link in links2:
    print(link)
    try:
        URL = f"https://www.dailymail.co.uk{link}"
        response = requests.get(URL) 
        soup = BeautifulSoup(response.content) 
        story = soup.find('div', itemprop="articleBody").find_all('p', class_=lambda x: x != 'imageCaption')
        url_dict = {}
        url_dict['link'] = link
        lines = []
        for line in story:
            sent = line.text
            lines.append(sent)
        url_dict['article'] = lines
        stories2.append(url_dict)
    except:
        print('error', link)

/femail/article-7727077/Kate-Middleton-reacts-Camila-Cabello-stealing-pencil-Buckingham-Palace.html
/femail/article-7722691/Gemma-Collins-named-making-biggest-fashion-faux-pas-DECADE-far.html
/femail/article-7722437/How-Archie-look-grows-up.html
/femail/article-7722387/Meghan-Markle-aware-pitted-against-future-queen-Kate-Middleton-source-claims.html
/femail/article-7722343/Meghan-Markle-Prince-Harry-say-Archie-eating-solids.html
/tvshowbiz/article-7721451/Olivia-Colman-looks-radiant-bejewelled-black-gown-presents-award-Best-Play.html
/news/article-7720725/Man-teenager-stabbed-car-crash-Manchester.html
/femail/article-7715019/Expert-suggests-Prince-William-Kates-stripped-staff-reason-absence.html
/femail/article-7714481/Season-3-Netflixs-Crown-unexpectedly-paid-tribute-Prince-George.html
/femail/article-7714195/Kate-Middleton-wrote-personal-thankyou-note-designer-Pakistan-wardrobe.html
/femail/article-7713919/Kate-Middleton-Prince-William-hosted-private-Tusk-Awards-reception-pulled-out.

/news/article-7659353/Harry-Meghan-visit-Westminster-Abbey-Field-Remembrance.html
/femail/article-7659999/Kate-Middleton-teams-Princess-Dianas-earrings-cobalt-fit-flare-dress-London-outing.html
/news/article-7659431/Prince-William-Kate-Middleton-meet-victims-Grenfell-Tower-fire.html
/news/article-7657033/ABCs-Amy-Robach-skips-work-heads-gym-following-explosive-hot-mic-revelation.html
/femail/article-7655017/As-Queen-bans-fur-outfits-different-approaches-taken-royals.html
/femail/article-7655109/Former-soldier-Ant-Middleton-claims-men-arent-allowed-manly-anymore.html
/tvshowbiz/article-7653987/Liam-Payne-feels-lucky-alive-reveals-perils-One-Direction-fame-nearly-killed-him.html
/femail/article-7651525/Queen-Elizabeth-II-not-buy-new-outfits-containing-real-fur-says-palace.html
/news/article-7651987/Amy-Robach-reveals-ABC-killed-Jeffrey-Epstein-expose-fears-Prince-William-Kate.html
/galleries/article-7642533/20-weird-facts-probably-didnt-know-British-Royal-family.html
/news/article-765112

/femail/article-7567983/Terminally-ill-boy-set-meet-Prince-Harry-Wills-told-tell-brother-likes-best.html
/debate/article-7568969/In-mums-footsteps-RICHARD-KAY-William-Kates-visit-Pakistan.html
/tvshowbiz/article-7567569/Vogue-Williams-looks-radiant-makes-stylish-exit-West-End-show.html
/femail/article-7566733/Delighted-father-Kate-brothers-French-fianc-e-says-genuine-love-story.html
/femail/article-7556921/Nine-complete-spring-racing-looks-100.html
/news/article-7565441/James-Middleton-reveals-considering-suicide-Duchess-accompanied-therapy.html
/tvshowbiz/article-7563435/Vogue-Williams-looks-radiant-stunning-swimsuits-promoting-tanning-range.html
/femail/article-7558385/How-polo-necks-Kate-Middletons-wardrobe-staple.html
/news/article-7560357/Drunken-thug-Thomas-Cook-flight-pulled-underwear-pushed-cabin-crew-plane-door.html
/femail/article-7555637/The-five-major-differences-Kate-Middleton-Meghan-Markles-style.html
/femail/article-7554391/Heirs-reality-TV-crown-Real-Housewives-Cheshire

/femail/article-7454701/William-Kate-work-director-Richard-Curtis-mental-health-project.html
/femail/article-7452357/Sophie-Wessex-Queens-favourite-royal-insider-claims.html
/femail/article-7451465/Kate-Middleton-opened-Nature-garden-praised-grandparents.html
/femail/article-7447981/Kate-Middleton-Meghan-Markle-love-royal-approved-stationer-Smythson.html
/galleries/article-7440591/From-taking-selfies-fighting-Afghanistan-times-Prince-Harry-broke-Royal-rules.html
/femail/article-7447187/Prince-Harry-Meghan-Markle-Prince-William-Kate-Middleton-mark-World-Suicide-Prevention-Day.html
/femail/article-7447233/Kate-Middleton-looks-elegant-arrives-Nature-play-garden-Wisley-today.html
/femail/article-7447163/Prince-William-reveals-Princess-Charlotte-loves-unicorns-calls-cool.html
/tvshowbiz/article-7445941/Downton-Abbeys-Laura-Carmichael-puts-leggy-display-sexy-LBD.html
/tvshowbiz/article-7445369/BRIAN-VINER-big-screen-version-Downton-Abbey.html
/tvshowbiz/article-7444769/Downton-Abbey-Movie-Tu

/news/article-7350433/Labour-gets-roasting-countryside-groups-declaring-war-grouse-shooting.html
/tvshowbiz/article-7348837/Donna-Air-seen-time-splitting-boyfriend-Ben-Carrington.html
/femail/article-7348673/Pippa-Middleton-sports-casual-look-jeans-shirt-stroll-son-Arthur-Chelsea.html
/news/article-7347463/Thousands-pensioners-fear-hoodwinked-closing-Post-Office-card-accounts.html
/femail/article-7345991/Meghan-Markle-victim-mum-shaming-PR-manager-claims.html
/news/article-7345843/Are-white-trainers-FORMAL-wear-People-following-celebrity-trends-say-etiquette-experts.html
/news/article-7345443/Duchess-Cambridges-old-Chelsea-flat-sold-70-000-1-95million-asking-price.html
/news/article-7345351/Oh-midge-wall-Balmoral-Queen-bracing-host-Andrew-Fergie.html
/tvshowbiz/article-7344955/TALK-TOWN-Countdown-presenter-Rachel-Riley-enjoys-romantic-stroll-husband-Pasha-Kovalev.html
/news/article-7345149/Scammers-use-Meghan-Markles-face-sell-diet-pills.html
/tvshowbiz/article-7343163/Donna-Air-splits

...

/news/article-5749973/Inside-Harry-Meghans-lavish-evening-wedding-reception-hosted-Prince-Charles.html
/femail/article-5749935/Kensington-Palace-updates-Twitter-profile-photo-include-Meghan-Markle.html
/news/article-5749899/Harry-Meghans-party-guests-entertained-DJ-stars-played-Pippas-wedding.html
/femail/article-5749281/Top-wedding-songs-Australians-guess-number-one.html
/femail/article-5749121/The-Royal-Wedding-hits-misses-LIZ-JONES.html
/news/article-5748979/Funniest-memes-Royal-Wedding.html
/news/article-5748861/How-Meghans-200-000-Givenchy-gown-compares-new-royal-laws.html
/news/article-5748651/Meghan-Harry-head-Frogmore-House-reception-royal-wedding.html
/tvshowbiz/article-5748359/JIM-SHELLEY-BBC-ITV-didnt-know-Royal-wedding-celebrity-event-not-Sky-nightmare.html
/news/article-5748193/Prince-Philip-Duke-Edinburgh-walks-unaided-Royal-wedding-weeks-hip-operation.html
/femail/article-5748085/Meghan-Markles-closest-girlfriends-support-wedding-day.html
/femail/fb-5748239/What-does-Meg

/femail/article-5673683/Meghan-Eugenie-driving-sales-replica-engagement-rings.html
/news/article-5672127/Prince-William-Kate-Middleton-mark-seventh-wedding-anniversary.html
/femail/article-5671923/Can-guess-girlfriend-daughter.html
/tvshowbiz/article-5669819/Laurence-Llewelyn-Bowen-jokes-neighbours-Prince-Charles-Camilla.html
/femail/article-5662327/How-Kates-jewellery-designer-royal-favourite.html
/tvshowbiz/article-5668397/Georgia-Toffolo-accentuates-svelte-frame-sleeveless-knit-night-out.html
/debate/article-5667019/Carole-Middleton-prevail-Charles.html
/news/article-5666301/How-Kate-Middleton-strong-links-Arthur-Charles-family.html
/femail/article-5666669/Mothers-comparing-post-birth-pics-Kate-Middletons.html
/debate/article-5666581/Prince-Louis-touching-tribute-colossus-empire-cruelly-cut-IRA.html
/news/article-5666317/Is-royal-baby-look-like.html
/news/article-5665913/Buckingham-Palace-launches-gold-gilded-Prince-Louis-mug.html
/femail/article-5665301/Kates-Lindo-Wing-dress-looks

/news/article-5591157/Hooded-man-throat-slit-gestures-home-woman-accusing-David-Matthews-rape.html
/tvshowbiz/article-5590439/Radio-host-Mel-Greig-shares-experience-noisy-neighbours-having-sex.html
/news/article-5589963/Im-no-fantasist-Pippa-Middletons-father-law-raped-want-justice.html
/money/markets/article-5587081/Babywear-brand-Jojo-Maman-Bebe-defies-gloom-High-Street-boosts-sales-20pc.html
/tvshowbiz/article-5586459/Vogue-Williams-covers-baby-bump-day-Spencer-Matthews.html
/news/article-5586353/Ouch-Dog-gets-face-quills-distinctly-spiky-encounter-GIANT-porcupine.html
/femail/article-5585231/Will-Erdem-design-Meghans-wedding-dress.html
/news/article-5584175/Second-woman-claim-Pippa-Middletons-father-law-raped-family-friend.html
/news/article-5582175/Pippa-Middleton-steps-time-father-law-charged.html
/tvshowbiz/article-5581481/Vogue-Williams-covers-baby-bump-fiance-Spencer-Matthews-father-faces-second-rape-probe.html
/news/article-5579823/Pippa-Middletons-father-law-SECOND-rape-prob

/femail/article-5414299/Whos-row-Queen.html
/news/article-5410109/Furious-couple-sue-wedding-photographer.html
/tvshowbiz/article-5410665/Now-Donna-Air-dates-pal-Pippas-brother-law.html
/femail/article-5408379/Did-Kates-1M-EMERALDS-signal-support-Times-Up.html
/tvshowbiz/article-5408283/Donna-Air-steps-new-man-Ben-Carrington.html
/tvshowbiz/article-5406373/BAFTAs-2018-Pregnant-Kate-DEFIES-Times-protest.html
/tvshowbiz/article-5404689/Survival-Fittest-Ryan-Cleary-voted-off.html
/tvshowbiz/article-5404359/Donna-Air-James-Middleton-split-reveals.html
/travel/article-5402997/Secrets-William-Harrys-African-sanctuary.html
/femail/article-5401475/FEMAIL-rounds-weeks-fashion-highlights.html
/femail/article-5401777/Links-young-royals-Chelsea-cast.html
/tvshowbiz/article-5400211/Georgia-Toff-Toffolo-puffs-cigarette-Chelsea.html
/tvshowbiz/article-5399741/Millie-Mackintosh-looks-chic-leopard-print-coat.html
/tvshowbiz/article-5392915/Survival-Fittest-James-Middleton-bursts-TEARS.html
/tvshowbiz/a

/tvshowbiz/article-5209295/Princess-Beatrices-mystery-date-revealed.html
/news/article-5207683/QUENTIN-LETTS-imagines-Christmas-Sandringham.html
/news/article-5205501/Hermes-Delivery-driver-appears-chuck-parcels-lorry.html
/femail/article-5196323/Duchess-Cambridges-119K-shopping-bill.html
/femail/article-5204149/From-stiff-schmaltzy-100-years.html
/sport/cricket/article-5204021/Sussex-fast-bowler-Archer-taking-Australia-storm.html
/news/article-5203465/Chris-Tarrant-claims-sober-drink-drive-probe.html
/tvshowbiz/article-5202529/Amy-Childs-tries-sell-calendar-celeb-boot-sale.html
/news/article-5201959/New-Prince-Harry-Meghan-Markle-photograph.html
/tvshowbiz/article-5201643/Kerry-Katona-makes-jubilant-return-panto-rehearsals.html
/news/article-5200677/Chris-Tarrant-appears-court-charged-drink-driving.html
/femail/article-5200527/Jodie-Kidd-goes-eco-friendly-Christmas.html
/dailymailtv/article-5200209/Exclusive-pics-FBI-agent-sent-F-Trump-texts.html
/femail/article-5198241/Royals-gather-

In [52]:
middleton_dm_stories = pd.DataFrame(stories2)

In [60]:
middleton_dm_stories_all = pd.merge(middleton_dm_stories, df2, on='link')

In [63]:
middleton_dm_stories_all = middleton_dm_stories_all[middleton_dm_stories_all.title.str.contains('Kate|Middleton')]

In [64]:
middleton_dm_stories_all.shape

(724, 5)

In [65]:
middleton_dm_stories_all.to_csv('middleton_dm_stories_Nov17to19.csv')

# The Sun

### Meghan Markle

Scraping The Sun's website's search feature for links to articles with the search term "Meghan Markle."

Scraped Nov, 27, 2019.

In [ ]:
markle_sunlinks = []

for i in range(1,650,1):
    try:
        URL = f"https://www.thesun.co.uk/page/{i}/?s=meghan+markle"
        response = requests.get(URL) 
        soup = BeautifulSoup(response.content) 
        hits = soup.find_all('div', class_='sun-row teaser')
        for hit in hits:
            link_dict = {}
            link_dict['date'] = (hit.find('div', class_='teaser-item').find('div', class_='search-date').text)
            link_dict['link'] = (hit.find('div', class_='teaser-item').find('a', class_='text-anchor-wrap').get('href'))
            link_dict['title'] = (hit.find('div', class_='teaser-item').find('a', class_='text-anchor-wrap').p.text.strip())
            markle_sunlinks.append(link_dict)
    except:
        print('error', i)

In [ ]:
df_sun = pd.DataFrame(markle_sunlinks)

In [ ]:
df_sun.shape

In [ ]:
df_sun.sample(5)

In [ ]:
df_sun.date = pd.to_datetime(df_sun.date)

In [ ]:
df_sun = df_sun[(df_sun.date  > '2017-11-27') & (df_sun.date < '2019-11-27')]

In [ ]:
df_sun.date.min()

In [ ]:
df_sun.date.max()

In [ ]:
df_sun = df_sun[df_sun.title.str.contains('Meghan|Markle')]

In [ ]:
df_sun.head()

In [ ]:
df_sun.to_csv('markle_sun_links_Nov17to19.csv')

In [ ]:
df_sun = pd.read_csv('tabloid_output/The_Sun/markle_sun_links_Nov17to19.csv')

In [ ]:
sun_links = df_sun.link.to_list()

In [ ]:
len(sun_links)

In [ ]:
stories = []
for link in sun_links:
    print(link)
    try:
        URL = link
        response = requests.get(URL) 
        soup = BeautifulSoup(response.content) 
        story = soup.find('div', class_="article__content").find_all('p', class_=None)
        story_dict = {}
        story_dict['link'] = link
        lines = []
        for line in story:
            sent = line.text.replace(u'\xa0', u' ')
            sent = sent.replace(u'\n', u' ')
            lines.append(sent)
        story_dict['article'] = lines
        try:
            story_dict['author'] = soup.find('div', class_="article__meta").find('a', class_="author").get('data-author')
        except:
            story_dict['author'] = soup.find('div', class_="article__meta").find('span', class_="article__author-name").text
        stories.append(story_dict)
    except:
        print('error', link)

In [ ]:
sun_stories = pd.DataFrame(stories)

In [ ]:
sun_stories.head()

In [ ]:
sun_stories.shape

In [ ]:
markle_sun = pd.merge(sun_stories, df_sun, on='link')

In [ ]:
# markle_sun = markle_sun[['article','author','link','date','title']]

In [ ]:
markle_sun.shape

In [ ]:
markle_sun

In [ ]:
markle_sun.to_csv('markle_sun_stories_Nov17to19.csv')

### Kate Middleton

Scraping The Sun's website's search feature for links to articles with the search term "Kate Middleton."

Scraped Nov. 27, 2019.

In [ ]:
middleton_sunlinks = []

for i in range(1,375,1):
    try:
        URL = f"https://www.thesun.co.uk/page/{i}/?s=kate+middleton"
        response = requests.get(URL) 
        soup = BeautifulSoup(response.content) 
        hits = soup.find_all('div', class_='sun-row teaser')
        for hit in hits:
            link_dict = {}
            link_dict['date'] = (hit.find('div', class_='teaser-item').find('div', class_='search-date').text)
            link_dict['link'] = (hit.find('div', class_='teaser-item').find('a', class_='text-anchor-wrap').get('href'))
            link_dict['title'] = (hit.find('div', class_='teaser-item').find('a', class_='text-anchor-wrap').p.text.strip())
            middleton_sunlinks.append(link_dict)
    except:
        print('error', i)

In [ ]:
middleton_sun = pd.DataFrame(middleton_sunlinks)

In [ ]:
len(middleton_sunlinks)

In [ ]:
middleton_sun.head()

In [ ]:
# middleton_sun.to_csv('middleton_sun_links.csv')

In [ ]:
middleton_sun.date = pd.to_datetime(middleton_sun.date)

In [ ]:
middleton_sun = middleton_sun[(middleton_sun.date  > '2017-11-27') & (middleton_sun.date < '2019-11-27')]

In [ ]:
middleton_sun.date.min()

In [ ]:
middleton_sun.date.max()

In [ ]:
middleton_sun = middleton_sun[middleton_sun.title.str.contains('Middleton|Kate|Duchess')]

In [ ]:
middleton_sun_links = middleton_sun.link.to_list()

In [ ]:
stories2 = []
for link in middleton_sun_links:
    print(link)
    try:
        URL = link
        response = requests.get(URL) 
        soup = BeautifulSoup(response.content) 
        story = soup.find('div', class_="article__content").find_all('p', class_=None)
        story_dict = {}
        story_dict['link'] = link
        lines = []
        for line in story:
            sent = line.text.replace(u'\xa0', u' ')
            sent = sent.replace(u'\n', u' ')
            lines.append(sent)
        story_dict['article'] = lines
        try:
            story_dict['author'] = soup.find('div', class_="article__meta").find('a', class_="author").get('data-author')
        except:
            story_dict['author'] = soup.find('div', class_="article__meta").find('span', class_="article__author-name").text
        stories2.append(story_dict)
    except:
        print('error', link)

In [ ]:
middleton_sun_stories = pd.DataFrame(stories2)

In [ ]:
middleton_sun_stories.shape

In [ ]:
middleton_sun_stories.head()

In [ ]:
middleton_sun.head()

In [13]:
middleton_sun_stories = pd.read_csv('middleton_sun_stories_Nov17to19.csv')

In [14]:
middleton_sun_links = pd.read_csv('middleton_sun_links.csv')

In [15]:
middleton_sun_stories = pd.merge(middleton_sun_stories, middleton_sun_links, on='link' )

In [16]:
middleton_sun_stories.to_csv('middleton_sun_stories_Nov17to19.csv')

# The Mirror

### Meghan Markle

In [ ]:
markle_mirrorlinks = []
headers = {'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0'}
for i in range(1,140,1):
    try:
        URL = f"https://www.mirror.co.uk/all-about/meghan-markle?pageNumber={i}"
        response = requests.get(URL, headers=headers) 
        soup = BeautifulSoup(response.content) 
        hits = soup.find_all('div', class_='teaser')
        for hit in hits:
            link_dict = {}
            link_dict['link'] = (hit.find('a', class_='headline').get('href'))
            link_dict['title'] = (hit.find('a', class_='headline').text.strip())
            markle_mirrorlinks.append(link_dict)
    except:
        print('error', i)

In [ ]:
markle_mirror_links = pd.DataFrame(markle_mirrorlinks)

In [ ]:
markle_mirror_links.to_csv('markle_mirror_links.csv')

In [ ]:
markle_mirror_links = pd.read_csv('markle_mirror_links.csv')

In [ ]:
markle_mirror_links = markle_mirror_links[['link','title']]

In [ ]:
markle_mirror_links.shape

In [ ]:
markle_mirror_links = markle_mirror_links[markle_mirror_links.title.str.contains('Meghan|Markle')]

In [ ]:
markle_mirror_links.shape

In [ ]:
mirror_links_markle = markle_mirror_links.link.to_list()

In [ ]:
headers = {'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0'}
stories3 = []

for link in mirror_links_markle:
    print(link)
    try:
        URL = link
        response = requests.get(URL, headers=headers) 
        soup = BeautifulSoup(response.content) 
        story = soup.find('div', class_="article-body").find_all('p', class_=None)
        story_dict = {}
        story_dict['link'] = link
        lines = []
        for line in story:
            sent = line.text
            sent = sent
            lines.append(sent)
        story_dict['article'] = lines
        try:
            story_dict['author'] = soup.find('div', class_="article-information").find('div', class_='author').find('a').text
        except:
            story_dict['author'] = soup.find('div', class_="article-information").find('div', class_='author').find('span').text
        story_dict['date'] = soup.find('div', class_="article-information").find('ul', class_='time-info').find('time').get('datetime')
        stories3.append(story_dict)
    except:
        print('error', link)

In [ ]:
df_6 = pd.DataFrame(stories3)

In [ ]:
df_6 = pd.merge(df_6, markle_mirror_links, on='link')

In [ ]:
df_6.date = df_6.date.str.replace('T', ' ')

In [ ]:
df_6.date = df_6.date.str.replace('Z', '')

In [ ]:
df_6.date = pd.to_datetime(df_6.date)

In [ ]:
df_6 = df_6[(df_6.date  > '2017-11-27') & (df_6.date < '2019-11-27')]

In [ ]:
df_6.date.max()

In [ ]:
df_6.date.min()

In [ ]:
df_6.to_csv('markle_mirror_stories_Nov17to19.csv')

### Kate Middleton

In [ ]:
middleton_mirrorlinks = []
headers = {'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0'}
for i in range(1,40,1):
    try:
        URL = f"https://www.mirror.co.uk/all-about/prince-william?pageNumber={i}"
        response = requests.get(URL, headers=headers) 
        soup = BeautifulSoup(response.content) 
        hits = soup.find_all('div', class_='teaser')
        for hit in hits:
            link_dict = {}
            link_dict['link'] = (hit.find('a', class_='headline').get('href'))
            link_dict['title'] = (hit.find('a', class_='headline').text.strip())
            middleton_mirrorlinks.append(link_dict)
    except:
        print('error', i)

In [ ]:
middleton_mirrorlinks2 = pd.DataFrame(middleton_mirrorlinks)

In [ ]:
middleton_mirrorlinks2 = middleton_mirrorlinks2[middleton_mirrorlinks2.title.str.contains('Kate|Middleton')]

In [ ]:
# middleton_mirrorlinks2.to_csv('middleton_mirror_links.csv')

In [ ]:
royal_mirrorlinks = []
headers = {'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0'}
for i in range(1,200,1):
    print(i)
    try:
        URL = f"https://www.mirror.co.uk/all-about/royal-family?pageNumber={i}"
        response = requests.get(URL, headers=headers) 
        soup = BeautifulSoup(response.content) 
        hits = soup.find_all('div', class_='teaser')
        for hit in hits:
            link_dict = {}
            link_dict['link'] = (hit.find('a', class_='headline').get('href'))
            link_dict['title'] = (hit.find('a', class_='headline').text.strip())
            royal_mirrorlinks.append(link_dict)
    except:
        print('error', i)

In [ ]:
royal_mirrorlinks2 = pd.DataFrame(royal_mirrorlinks)

In [ ]:
royal_mirrorlinks2.to_csv('royal_mirror_links.csv')

In [ ]:
royal_mirrorlinks2 = royal_mirrorlinks2[royal_mirrorlinks2.title.str.contains('Kate|Middleton')]

In [ ]:
middleton_mirror_links = middleton_mirrorlinks2.append(royal_mirrorlinks2)

In [ ]:
middleton_mirror_links.duplicated().value_counts()

In [ ]:
middleton_mirror_links = middleton_mirror_links.drop_duplicates()

In [ ]:
middleton_mirror_links.head()

In [ ]:
middleton_mirror_links.to_csv('allmiddleton_mirror_links.csv')

In [ ]:
mirror_links_middleton = middleton_mirror_links.link.to_list()

In [ ]:
headers = {'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0'}
stories4 = []

for link in mirror_links_middleton:
    print(link)
    try:
        URL = link
        response = requests.get(URL, headers=headers) 
        soup = BeautifulSoup(response.content) 
        story = soup.find('div', class_="article-body").find_all('p', class_=None)
        story_dict = {}
        story_dict['link'] = link
        lines = []
        for line in story:
            sent = line.text
            sent = sent
            lines.append(sent)
        story_dict['article'] = lines
        try:
            story_dict['author'] = soup.find('div', class_="article-information").find('div', class_='author').find('a').text
        except:
            story_dict['author'] = soup.find('div', class_="article-information").find('div', class_='author').find('span').text
        story_dict['date'] = soup.find('div', class_="article-information").find('ul', class_='time-info').find('time').get('datetime')
        stories4.append(story_dict)
    except:
        print('error', link)

In [ ]:
stories_df = pd.DataFrame(stories4)

In [ ]:
stories_df.shape

In [ ]:
middleton_stories_all = pd.merge(stories_df, middleton_mirror_links, on='link')

In [ ]:
middleton_stories_all.to_csv('middleton_mirror_stories_Nov17to19.csv')

In [10]:
pd.read_csv('middleton_mirror_stories_Nov17to19.csv').shape

(772, 6)